To generate files necessary to run cluster.py

In [2]:
import pandas as pd
import numpy as np
import pickle

To create fingerprint dictionary of all compounds in the dataset (morganfp_2_1024_dict_PDBBIND2018+DTC.p): need to create a table with all compounds in the dataset /n/groups/alquraishi/Changchang/Datasets/PDBBind2018/PDBBIND2018+DTC_compounds.csv and run lgd_morgan_mtx_PDBBIND2018+DTC.py to map morgan fingerprints to it

In [4]:
pdbbind_dtc = pd.read_csv('../../resources/PDBBIND2018_DTC_1127.csv',
                         dtype={'ENTREZ_GENE_ID':object})
unique_data = pd.read_csv('../../resources/PDBBIND2018_DTC_1127_unique.csv', 
                           dtype={'ENTREZ_GENE_ID':object})

In [3]:
# clean up pdbbind by giving all kinases gene id based on their Uniprot Entry
extended_kinome = pd.read_csv('../../resources/the_697_extended_kinome_v1_annotated.csv', 
                          dtype={'gene_id':object})

In [4]:
unique_data = pdbbind_dtc.drop_duplicates(subset=['ENTREZ_GENE_ID', 'INCHIKEY_DESALT'])
unique_data['pair'] = unique_data['ENTREZ_GENE_ID']+'|'+unique_data['INCHIKEY_DESALT']

/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


To create kinase_bound_per_cmpd dictionary

In [6]:
compound_inchikeys = unique_data['INCHIKEY_DESALT'].drop_duplicates().tolist()

kinase_bound_per_cmpd = {'num':{}, 'kinase':{}}
for acmpd in compound_inchikeys:
    kinase_bound_per_cmpd['kinase'][acmpd] = unique_data.loc[unique_data['INCHIKEY_DESALT']==acmpd]['ENTREZ_GENE_ID'].tolist()
    kinase_bound_per_cmpd['num'][acmpd] = len(kinase_bound_per_cmpd['kinase'][acmpd])

In [7]:
with open('kinase_bound_per_cmpd.p', 'wb') as handle:
    pickle.dump(kinase_bound_per_cmpd, handle)

calculate the kinase distance matrix

In [8]:
uniprotnametogeneid = {}
for idx, arow in extended_kinome.iterrows():
    uniprotnametogeneid[arow['Uniprot Entry name']] = arow['gene_id']
uniprotnametogeneid['GRK1_HUMAN'] = '6011'
uniprotnametogeneid['FCSK_HUMAN'] = '197258'

In [51]:
# distance dictionary is created via multiple sequence alignment of all extended kinome kinases with UniRef90 using 
# jackhmmer
tgt_dist_dict_by_uniprotname_v4 = pickle.load(open('jackhmmer_dist_dict_v4.p', 'rb')) 

In [52]:
# create distance metric by gene name. For multi-domain proteins, this will record the distance of the domain 
# more similar in sequence so that clustering is not overoptimistic

tgt_dist_dict_by_gene_v4 = {}
for akey, aval in tgt_dist_dict_by_uniprotname_v4.items():
    pair = sorted(list(akey))
    new_key = []
    for i in pair:
        if 'HUMAN_2' in i:
            new_key.append(uniprotnametogeneid[i[:-2]])
        else:
            new_key.append(uniprotnametogeneid[i])
    new_key = frozenset(new_key)
    
    try:
        tgt_dist_dict_by_gene_v4[new_key] = min(aval, tgt_dist_dict_by_gene_v4[new_key])
    except KeyError:
        tgt_dist_dict_by_gene_v4[new_key] = aval

In [54]:
total_dataset_kinases = pdbbind_dtc.loc[pdbbind_dtc['PKL']]['ENTREZ_GENE_ID'].astype(str).drop_duplicates().tolist()
# check which kinases not included in similarity dict. This is only because 10454 and 5783 are actually phosphatases
# 150094 is SIK1 which will be addressed later
for akin in total_dataset_kinases:
    try:
        tgt_dist_dict_by_gene_v4[frozenset([akin])]
    except KeyError:
        try:
            int(akin)
            print(akin)
        except ValueError:
            continue

150094


In [59]:
# add SIK1 150094
tgt_dist_dict_by_gene_v4_parsed = {}
for akey, aval in tgt_dist_dict_by_gene_v4.items():
    old_key = list(akey)
    tgt_dist_dict_by_gene_v4_parsed[frozenset(old_key)] = aval
    if '102724428' in old_key:
        new_key = ['150094' if x=='102724428' else x for x in old_key]
        tgt_dist_dict_by_gene_v4_parsed[frozenset(new_key)] = aval

In [8]:
kinase_pdbproteins = pdbbind_dtc['ENTREZ_GENE_ID'].astype(str).drop_duplicates().tolist()

In [61]:
# complete the distance matrix by assigning not aligned kinases to 0
# Assign sequence similarity non-kinases in PDBBind to 0

tgt_dist_dict_by_gene_finish = {}
for akin in kinase_pdbproteins:
    for bkin in kinase_pdbproteins: 
        try:
            tgt_dist_dict_by_gene_finish[frozenset([str(akin), str(bkin)])] = tgt_dist_dict_by_gene_v4_parsed[frozenset([akin, bkin])]
        except KeyError:
            if akin == bkin:
                tgt_dist_dict_by_gene_finish[frozenset([str(akin), str(bkin)])] = 0.
            else:
                tgt_dist_dict_by_gene_finish[frozenset([str(akin), str(bkin)])] = 100.

In [62]:
with open('tgt_dist_dict_by_gene_v4.p', 'wb') as handle:
    pickle.dump(tgt_dist_dict_by_gene_finish, handle)

To create cmpd_bound_per_kinase dictionary

In [9]:
cmpd_bound_per_kinase = {'num':{}, 'compound':{}}
for akinase in kinase_pdbproteins:
    cmpd_bound_per_kinase['compound'][akinase] = unique_data.loc[unique_data['ENTREZ_GENE_ID']==akinase]['INCHIKEY_DESALT'].tolist()
    cmpd_bound_per_kinase['num'][akinase] = len(cmpd_bound_per_kinase['compound'][akinase])

In [10]:
with open('cmpd_bound_per_kinase.p', 'wb') as handle:
    pickle.dump(cmpd_bound_per_kinase, handle)

Now identify pairs that would go into validation/test set. To have a fair representation of all affinities, I will divide the dataset into nonbinders, low affinities, high affinities, and crystal structures. 

In [5]:
pkl_pairs = unique_data.loc[unique_data['PKL']]

There are also kinase pairs where the ligands have metals or are peptides ('mer'). Because ligands in the docked structures do not contain metal, this might be a way for the model to differentiate between crystal and docked structures. Therefore I will only include pairs where the ligand do not have metals for training. xtal_lgd_clean contains only the pairs where are not peptides or metals

In [16]:
xtal_lgd_clean=pd.read_csv('../../resources/PDBBIND_ligand_cleaned.csv')

/Users/cl321/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


The docked pairs will be divided by their affinity: high, low and nonbinding

In [24]:
highaff = pkl_pairs.loc[(pkl_pairs['RESULT_VALUE_min']<-4)&(pkl_pairs['dataset'].isin(['KI', 'KD']))]
lowaff = pkl_pairs.loc[(pkl_pairs['RESULT_VALUE_min']>=-4)&(
    pkl_pairs['RESULT_VALUE_min']<np.log(10))&(
    pkl_pairs['dataset'].isin(['KI', 'KD']))]
nonbinding=pkl_pairs.loc[(pkl_pairs['RESULT_VALUE_min']>=np.log(10))&(pkl_pairs['dataset'].isin(['KI', 'KD']))]

In [25]:
xtalkinase_pairs = xtal_lgd_clean['pair'].tolist()
highaff_pairs = highaff['pair'].tolist()
lowaff_pairs = lowaff['pair'].tolist()
nonbinding_pairs = nonbinding['pair'].tolist()

Now identify pairs for pretraining. These will be non-kinase pairs

In [29]:
pdbbind_raw = unique_data.loc[unique_data['dataset']=='PDBBIND']
pdbbind_raw = pdbbind_raw.merge(PDBBIND[['PDB_CODE', 'ligand']])
pdbbind_nonkinase = pdbbind_raw.loc[(~pdbbind_raw['PKL'])|(pdbbind_raw['ligand'].str.contains('mer'))]
pdbbind_nonkinase_pairs = pdbbind_nonkinase['pair'].tolist()

All the kinases with PKL fold are included in validation/test

In [32]:
docked_kinases = pkl_pairs['ENTREZ_GENE_ID'].drop_duplicates().tolist()

Lastly all the pairs in the PDBBIND+DTC dataset are in the training/validation/test set

In [33]:
merge_table_pairs = unique_data['pair'].tolist()

save everything 

In [34]:
len(pdbbind_dtc['INCHIKEY_DESALT'].drop_duplicates().tolist())

24433

In [35]:
np.savetxt('PDBBIND+DTC_pairs.txt', merge_table_pairs, fmt='%s')
np.savetxt('xtal-kinase_pairs.txt', xtalkinase_pairs, fmt='%s')
np.savetxt('highaff_pairs.txt', highaff_pairs, fmt='%s')
np.savetxt('lowaff_pairs.txt', lowaff_pairs, fmt='%s')
np.savetxt('nonbinding_pairs.txt', nonbinding_pairs, fmt='%s')
np.savetxt('xtal-nonkinase_pairs.txt', pdbbind_nonkinase_pairs, fmt='%s')
np.savetxt('docked_kinases.txt', docked_kinases, fmt='%s')